In [1]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
from imutils.video import VideoStream
from imutils.video import FPS
import super_gradients
from super_gradients.common.object_names import Models
from super_gradients.training import Trainer
from super_gradients.training import training_hyperparams
from super_gradients.training.metrics.classification_metrics import Accuracy, Top5
from super_gradients.training.utils.early_stopping import EarlyStop
from super_gradients.training import models
from super_gradients.training.utils.callbacks import Phase
from super_gradients.training.utils.checkpoint_utils import load_checkpoint_to_model

import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset
import json
import os
from PIL import Image
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from glob import glob
# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px

import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

import plotly.io as pio
print(pio.renderers)
import gc
import requests
import torch
from PIL import Image
# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px

import seaborn as sns

The console stream is logged into C:\Users\Naufal\sg_logs\console.log


[2023-08-14 21:00:13] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-08-14 21:00:15] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-08-14 21:00:32] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-08-14 21:00:32] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-08-14 21:00:32] WARNING - export.py - Failed to import pytorch_quantization
[2023-08-14 21:00:32] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-08-14 21:00:32] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.
[2023-08-14 21:00:32] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command


		– MATPLOTLIB VERSION: 3.6.3


In [2]:
class config:
    # specify the paths to datasets

    TRAIN_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/train"
    TEST_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/test"
    VAL_DIR = '/kaggle/input/fruit-and-vegetable-image-recognition/validation'

    # set the input height and width
    INPUT_HEIGHT = 256
    INPUT_WIDTH = 256

    # set the input heig/ht and width
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    
    BATCH_SIZE = 128
    MODEL_NAME = 'regnetY800'
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    TRAINING_PARAMS = 'training_hyperparams/default_train_params'
    


    CHECKPOINT_DIR = 'checkpoints'
    

In [5]:
import cv2
import torch
from super_gradients.training  import models
from super_gradients.common.object_names import Models

#Load weight
model = models.get(config.MODEL_NAME, num_classes = 36, checkpoint_path="model.pth" )

[2023-08-14 21:01:57] INFO - checkpoint_utils.py - Successfully loaded model weights from model.pth checkpoint.


In [6]:
#all labels
class_names = ['pomegranate', 'pineapple', 'kiwi' ,'peas', 'lettuce', 'carrot' ,'mango',
 'paprika', 'raddish', 'apple', 'garlic', 'cauliflower', 'banana' ,'beetroot',
 'chilli pepper', 'spinach', 'watermelon', 'potato', 'turnip', 'ginger',
 'sweetcorn', 'lemon', 'eggplant', 'sweetpotato', 'capsicum', 'cucumber',
 'jalepeno', 'corn', 'onion', 'soy beans', 'bell pepper', 'orange', 'cabbage',
 'tomato', 'pear', 'grapes']



In [ ]:
#Pipeline for image preprocessing
transform = transforms.Compose([
    transforms.Resize(size=(config.INPUT_HEIGHT,config.INPUT_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.IMAGENET_MEAN,
                            std=config.IMAGENET_STD),
    ])

def predict(img):
    color_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_converted)
    transformed_image = transform(pil_image)
    return transformed_image

In [ ]:
img = cv2.imread()

In [7]:
cap = cv2.VideoCapture(0)

while True:
    # Capture an image from the camera
    res, frame = cap.read()
    
    # Preprocess the image
    color_converted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_converted)
    transformed_image = transform(pil_image)
    model.to(config.DEVICE)
    model.eval()
    with torch.inference_mode():
        # add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = transformed_image.unsqueeze(dim=0)

        # make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(config.DEVICE))
    # Probabilities
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

  

    # Classify the image
    height, width , channel = frame.shape
    sub_img = frame[0:int(height/6),0:int(width)]
    black_rect = np.ones(sub_img.shape, dtype=np.uint8)*0
    res = cv2.addWeighted(sub_img, 0.77, black_rect,0.23, 0)
    FONT = cv2.FONT_HERSHEY_SIMPLEX
    FONT_SCALE = 0.8
    FONT_THICKNESS = 2
    lable_color = (10, 10, 255)
    lable = "Realtime Fruit Classification"
    lable_dimension = cv2.getTextSize(lable,FONT ,FONT_SCALE,FONT_THICKNESS)[0]
    textX = int((res.shape[1] - lable_dimension[0]) / 2)
    textY = int((res.shape[0] + lable_dimension[1]) / 2)
    cv2.putText(res, "Label: {}".format(class_names[target_image_pred_label]), (0,textY+22+5), FONT,0.7, lable_color,2)
    lable_violation = 'Prob: {}'.format(str(target_image_pred_probs.max())+ "%")
    violation_text_dimension = cv2.getTextSize(lable_violation,FONT,FONT_SCALE,FONT_THICKNESS )[0]
    violation_x_axis = int(res.shape[1]- violation_text_dimension[0])
    cv2.putText(res, lable_violation, (violation_x_axis,textY+22+5), FONT,0.7, lable_color,2)
    frame[0:int(height/6),0:int(width)] =res
    cv2.imshow("Sugeng Classification", frame)

    # Press `q` to quit
    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()